In [3]:
import requests
import bs4
from datetime import datetime


In [68]:
r = requests.get("https://en.wikipedia.org/wiki/Category:Rare_diseases")

In [69]:
soup = bs4.BeautifulSoup(r.content, 'html.parser')

In [70]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Rare diseases - Wikipedia, the free encyclopedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   window.RLQ = window.RLQ || []; window.RLQ.push( function () {
mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Category:Rare_diseases","wgTitle":"Rare diseases","wgCurRevisionId":649465686,"wgRevisionId":649465686,"wgArticleId":1116081,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category with local link same as on Wikidata","Diseases and disorders"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"

In [71]:

last_mod = soup.find(id='footer-info-lastmod').text.split('on ')[-1].split(', at')[0]
print(last_mod)
last_mod_date = datetime.strptime(last_mod, '%d %B %Y')
print(last_mod_date)

2 March 2015
2015-03-02 00:00:00


In [89]:
import re
from itertools import chain
categories = soup.find(id="mw-pages").find_all(class_="mw-category-group")
links = list(chain(*[grp.find_all('a') for grp in categories]))

next_page = list(filter(lambda x: x.text == "next page", soup.find(id="mw-pages").find_all('a')))[0]
next_page.attrs['href']

'/w/index.php?title=Category:Rare_diseases&pagefrom=Fitzsimmons-Guilbert+syndrome%0AFitzsimmons%E2%80%93Guilbert+syndrome#mw-pages'

In [162]:
import re
from os.path import join
def get_info(uri, article=True):
    req = requests.get(uri)
    soup = bs4.BeautifulSoup(req.content, 'html.parser')
    info_ = {}
    info_['title'] = get_title(soup)
    info_['last_modified'] = get_last_modified(soup)
    #info_['category'] = get_category(soup)
    info_['content'] = get_content(soup, article)
    

def get_title(article_soup):
    # I think this will always work
    return soup.find(id='firstHeading').text

def get_last_modified(article_soup):
    last_mod_text = soup.find(id='footer-info-lastmod').text
    # fragile af
    last_mod_date = last_mod_test.split('on ')[-1].split(', at')[0]
    return datetime.strptime(last_mod_date, '%d %B %Y')

def make_link(partial_link):
    return ("http://en.wikipedia.org" + partial_link)

def get_next_page(category_soup):
    next_links = list(filter(lambda x: x.text == "next page", category_soup.find(id="mw-pages").find_all('a')))
    if len(next_links) == 0:
        return None
    else:
        return make_link(next_links[0].attrs['href'])
    
def get_links(category_soup):
    categories = category_soup.find(id="mw-pages").find_all(class_="mw-category-group")
    links = list(chain(*[grp.find_all('a') for grp in categories]))
    links = [make_link(link.attrs['href']) for link in links]
    return links
    
def get_content(article_soup, isCategory, maxlinks=None):
    # this is janky
    if isCategory:
        links = []
        page_counter = 0
        cur_soup = article_soup
        while (True):
            page_counter += 1
            links = list(chain(links, get_links(cur_soup)))
            if (maxlinks is not None) and len(links) >= maxlinks:
                links = links[:maxlinks]
                break
            next_page = get_next_page(cur_soup)
            if next_page is None:
                break
            else:
                req = requests.get(next_page)
                cur_soup = bs4.BeautifulSoup(req.content, 'html.parser')
        return links
        # first find all the article links
        # then if thereis a next page link, follow it, and so on
    else:
        # For article scraping a lot of times it should be sufficient to just find the
        # 'mw-content-text' id and find all the 'p' inside of it. We ideally don't want
        # to include the References section, as it usually contains something that gives
        # away the category. This usually works because the references section will have
        text_before_ref = soup.find(id='mw-content-text').get_text().split('References[edit]')[0]
        text_final = re.sub('\[(?:\d*)\]|\[(?:edit)\]','', text_before_ref)
        return text_final
    


In [161]:
get_content(soup, True)

http://en.wikipedia.org/w/index.php?title=Category:Rare_diseases&pagefrom=Microvillous+inclusion+disease#mw-pages
http://en.wikipedia.org/w/index.php?title=Category:Rare_diseases&pagefrom=Syrinx+%28Medicine%29%0ASyrinx+%28medicine%29#mw-pages
None


['http://en.wikipedia.org/wiki/Rare_disease',
 'http://en.wikipedia.org/wiki/13q_deletion_syndrome',
 'http://en.wikipedia.org/wiki/2-Hydroxyglutaric_aciduria',
 'http://en.wikipedia.org/wiki/3C_syndrome',
 'http://en.wikipedia.org/wiki/3q29_microdeletion_syndrome',
 'http://en.wikipedia.org/wiki/48,_XXXX',
 'http://en.wikipedia.org/wiki/49,_XXXXX',
 'http://en.wikipedia.org/wiki/8p23.1_duplication_syndrome',
 'http://en.wikipedia.org/wiki/Aarskog%E2%80%93Scott_syndrome',
 'http://en.wikipedia.org/wiki/ABCD_syndrome',
 'http://en.wikipedia.org/wiki/Abdominal_epilepsy',
 'http://en.wikipedia.org/wiki/Abetalipoproteinemia',
 'http://en.wikipedia.org/wiki/Ablepharon_macrostomia_syndrome',
 'http://en.wikipedia.org/wiki/Absent_adrenal_gland',
 'http://en.wikipedia.org/wiki/Acanthamoeba_keratitis',
 'http://en.wikipedia.org/wiki/Acheiropodia',
 'http://en.wikipedia.org/wiki/Achromatopsia',
 'http://en.wikipedia.org/wiki/Acrocallosal_syndrome',
 'http://en.wikipedia.org/wiki/Acrodysostosis',

In [158]:
req = requests.get("http://en.wikipedia.org/w/index.php?title=Category:Rare_diseases&pagefrom=Fitzsimmons-Guilbert+syndrome%0AFitzsimmons%E2%80%93Guilbert+syndrome#mw-pages")
get_next_page(bs4.BeautifulSoup(req.content, 'html.parser'))

'http://en.wikipedia.org/w/index.php?title=Category:Rare_diseases&pagefrom=Microvillous+inclusion+disease#mw-pages'